In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
from collections import Counter

In [3]:
import nltk

In [105]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/tron/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
val = pd.read_pickle('pre_data.pkl')

In [24]:
val.shape

(121512, 3)

In [22]:
val.head()

,image_id,question,answer
0,350623,What is the table made of?,wood
1,350623,Is the food napping on the table?,no
2,350623,What has been upcycled to make lights?,kettles
3,8647,Is this an Spanish town?,no
4,8647,Are there shadows on the sidewalk?,yes


In [10]:
def get_ans_vocab(data,col_name,max_val = 1000):
    import operator
    from collections import defaultdict
    freq = defaultdict(int)
    b=10
    for line in (data[col_name].unique()):
        for word in nltk.word_tokenize(str(line)):
            freq[word.lower()]+=1

    sort_freq = sorted(freq.items(),key=operator.itemgetter(1),reverse=True)[0:max_val]
    top_answers, top_fq = zip(*sort_freq)
    return top_answers

In [11]:
# def get_ques_vocab():
ques = get_ans_vocab(val,'question')

In [12]:
ans = get_ans_vocab(val, 'answer')

# Model designing

In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, Flatten, Embedding, Merge

In [ ]:
left_vgg = Sequential()
left_vgg.add(Dense(300, input_dim=4096, activation='relu'))

right_vgg = Sequential()
right_vgg.add(Dense(300, input_dim=4096, activation='relu'))

centre_w2vec = Sequential()
centre_w2vec.add(Dense(500,input_dim=300))

merge_layer = Merge([left_vgg,centre_w2vec,right_vgg], mode='concat')

lstm_model = Sequential()
lstm_model.add(merge_layer)
lstm_model.add(Dropout(dropout_rate))
lstm_model.add(LSTM(1000, input_shape=(1+1+500,300)))
lstm_model.add(Dense(1000, activation='softmax'))

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
lstm_model.fit([input_data_1, input_data_2, input_data_3], targets)

# Word2vec



In [7]:
val.columns

Index([u'image_id', u'question', u'answer'], dtype='object')

In [13]:
ques_list = list(val.question.values)

In [22]:
# ques_list

In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [53]:
from keras.preprocessing.text import base_filter
from keras.utils import np_utils

In [76]:
max_word = 2000
max_seq = 40

In [45]:
d = [s.encode('ascii') for s in ques_list]

In [50]:
print ques_list[1:5]

[u'Is the food napping on the table?', u'What has been upcycled to make lights?', u'Is this an Spanish town?', u'Are there shadows on the sidewalk?']


In [77]:
tokenizer = Tokenizer(nb_words=max_word, filters=base_filter(), lower=True, split=" ")
tokenizer.fit_on_texts(d)
sequences = tokenizer.texts_to_sequences(d)

word_index = tokenizer.word_index

data = pad_sequences(sequences, maxlen=max_seq)
# labels = np_utils.to_categorical(np.asarray())
print('Shape of data tensor:', data.shape)
# print('Shape of label tensor:', labels.shape)

('Shape of data tensor:', (121512, 40))


In [78]:
word_index['napping']

3175

In [79]:
data[1,:]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        2,  1, 49,  7,  1, 67], dtype=int32)

In [80]:
data.shape[0]

121512